In [1]:
import numpy as np
import pandas as pd


In [4]:
data=pd.read_csv('data/house_data.csv')


In [ ]:
data.sample(5)

In [ ]:
data.isna().sum()

In [ ]:
data.info()

In [ ]:
#Printing the unique values in each column
for columns in data:
    print(data[columns].value_counts())
    print("*"*30)
    

In [ ]:
#Dropping the columns which are not required
data.drop(columns=['area_type','availability','society','balcony'],inplace=True)

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data['location'].value_counts()

In [ ]:
#filling the missing values in the location column with the most frequent value
data['location']=data['location'].fillna('Whitefield')

In [ ]:
data['size'].value_counts()

In [ ]:
data['size']=data['size'].fillna('2 BHK')

In [ ]:
data['bath'].value_counts()

In [ ]:
data['bath']=data['bath'].fillna(data['bath'].median())

In [ ]:
#Converting the size column to integer
data['bhk']=data['size'].str.split().str.get(0).astype(int)

In [ ]:
#Finding outliers in the bhk column
data[data['bhk']>20]

In [ ]:
data['total_sqft'].unique()

In [ ]:
#Function to convert the range values in the total_sqft column to float
def rangeconverter(x):
    temp=x.split('-')
    if len(temp)==2:
        return (float(temp[0])+float(temp[1]))/2
    try:
        return float(x)
    except:
        return None

In [ ]:
data['total_sqft']=data['total_sqft'].apply(rangeconverter)

In [ ]:
data.sample(5)

### Adding price_per_sqft column

In [ ]:
data['price_per_sqft']=round(data['price']*100000/data['total_sqft'],2)

In [ ]:
data['price_per_sqft']

In [ ]:
data.describe()

In [ ]:
#Stripping the location column
data['location']=data['location'].apply(lambda x: x.strip())
location_counts=data['location'].value_counts()

In [ ]:
#Finding location with less than 10 data points and replacing them with 'other'
location_counts_less10=location_counts[location_counts<=10]
data['location']=data['location'].apply(lambda x: 'other' if x in location_counts_less10 else x)

In [ ]:
data['location'].sample(5)

### Outlier detection and removal

In [43]:
data.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,13274.000000,13320.000000,13320.000000,13320.000000,1.327400e+04
mean,1559.626694,2.688814,112.565627,2.802778,7.907501e+03
std,1238.405258,1.338754,148.971674,1.294496,1.064296e+05
min,1.000000,1.000000,8.000000,1.000000,2.678300e+02
25%,1100.000000,2.000000,50.000000,2.000000,4.266868e+03
50%,1276.000000,2.000000,72.000000,3.000000,5.434305e+03
75%,1680.000000,3.000000,120.000000,3.000000,7.311747e+03
max,52272.000000,40.000000,3600.000000,43.000000,1.200000e+07


In [44]:
(data['total_sqft']/data['bhk']).describe()

count    13274.000000
mean       575.074878
std        388.205175
min          0.250000
25%        473.333333
50%        552.500000
75%        625.000000
max      26136.000000
dtype: float64

In [45]:
#Selecting the data points where the total_sqft/bhk is greater than 300
data=data[((data['total_sqft']/data['bhk'])>=300)]
data.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,12530.000000,12530.000000,12530.000000,12530.000000,12530.000000
mean,1594.564544,2.559537,111.382401,2.650838,6303.979369
std,1261.271296,1.077938,152.077329,0.976678,4162.238008
min,300.000000,1.000000,8.440000,1.000000,267.830000
25%,1116.000000,2.000000,49.000000,2.000000,4210.530000
50%,1300.000000,2.000000,70.000000,3.000000,5294.120000
75%,1700.000000,3.000000,115.000000,3.000000,6916.670000
max,52272.000000,16.000000,3600.000000,16.000000,176470.590000


In [46]:
data['price_per_sqft'].describe()

count     12530.000000
mean       6303.979369
std        4162.238008
min         267.830000
25%        4210.530000
50%        5294.120000
75%        6916.670000
max      176470.590000
Name: price_per_sqft, dtype: float64

In [47]:
#Function to remove outliers in the price_per_sqft column
def remove_outliers_sqft(df):
    df_out=pd.DataFrame()
    #Grouping the data by location
    for key,subdf in df.groupby('location'):
        m=np.mean(subdf['price_per_sqft'])
        s=np.std(subdf['price_per_sqft'])

        #Selecting the data points where the price_per_sqft is within one standard deviation
        reduced_df=subdf[(subdf['price_per_sqft']>(m-s)) & (subdf['price_per_sqft']<=(m+s))]
        df_out=pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out



In [48]:
data=remove_outliers_sqft(data)
data.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,10301.000000,10301.000000,10301.000000,10301.000000,10301.000000
mean,1508.440608,2.471702,91.286372,2.574896,5659.062877
std,880.694214,0.979449,86.342786,0.897649,2265.774795
min,300.000000,1.000000,10.000000,1.000000,1250.000000
25%,1110.000000,2.000000,49.000000,2.000000,4244.900000
50%,1286.000000,2.000000,67.000000,2.000000,5175.600000
75%,1650.000000,3.000000,100.000000,3.000000,6428.570000
max,30400.000000,16.000000,2200.000000,16.000000,24509.800000


In [49]:
#function to remove outliers in the bhk column
def remove_outliers_bhk(df):
    #Array to store the indices of the data points to be removed
    exclude_indices=np.array([])
    for location,loc_df in df.groupby('location'):
        bhk_stats={}
        for bhk,bhk_df in loc_df.groupby('bhk'):

            #Calculating the mean and standard deviation of the price_per_sqft column for each bhk
            bhk_stats[bhk]={
                'mean':np.mean(bhk_df['price_per_sqft']),
                'std':np.std(bhk_df['price_per_sqft']),
                'count':bhk_df.shape[0]
            }
        for bhk,bhk_df in loc_df.groupby('bhk'):
            stats=bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices=np.append(exclude_indices,bhk_df[bhk_df['price_per_sqft']<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')


In [50]:
data=remove_outliers_bhk(data)
data.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,7361.000000,7361.000000,7361.000000,7361.000000,7361.000000
mean,1496.942529,2.448173,99.093113,2.500611,6127.640187
std,865.781990,1.011515,93.336841,0.929312,2408.997236
min,300.000000,1.000000,10.000000,1.000000,1300.000000
25%,1096.000000,2.000000,50.000000,2.000000,4600.000000
50%,1260.000000,2.000000,73.200000,2.000000,5680.000000
75%,1680.000000,3.000000,113.000000,3.000000,6896.550000
max,30000.000000,16.000000,2200.000000,16.000000,24509.800000


In [51]:
data.drop(columns=['size','price_per_sqft'],inplace=True)

### Clean and refined data

In [52]:
data.sample(5)

,location,total_sqft,bath,price,bhk
1744,Devanahalli,1360.0,2.0,65.0,2
10131,other,2400.0,3.0,185.0,4
6286,Sarjapur Road,1862.0,4.0,110.0,3
2382,Frazer Town,2560.0,3.0,288.0,3
10116,other,1950.2,3.0,193.0,3


In [53]:
data.to_csv('cleaned_data.csv',index=False)

In [55]:
X=data.drop(columns=['price'])
y=data['price']

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [57]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [58]:
print(X_train.shape)
print(X_test.shape)

(5888, 4)
(1473, 4)


### Applying linear regression

In [59]:
#Creating a column transformer to one hot encode the location column and scale the other columns
column_trans=make_column_transformer((OneHotEncoder(sparse=False),['location']),remainder='passthrough')


In [60]:
scalar=StandardScaler()

In [64]:
lnr=LinearRegression()

In [65]:
pipe=make_pipeline(column_trans,scalar,lnr)

In [66]:
pipe.fit(X_train,y_train)

c:\Users\Dell\miniconda3\envs\fellowship\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(sparse=False),
                                                  ['location'])])),
                ('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [67]:
y_pred=pipe.predict(X_test)

In [69]:
r2_score(y_test,y_pred)

0.8464673375029677

In [70]:
import pickle

In [72]:
pickle.dump(pipe,open('./models/prediction.pkl','wb'))